In [27]:
import glob
from GSC_API_extract import granular_extract
from insertion_csv_to_db import get_files, insert_files
from authorize_creds import authorize_creds
from datetime import date, timedelta, datetime
import mysql.connector
import pandas as pd
######################storage variables for database and desktop#######################################################
#database variables
#The searchType you input into a table should match the table name

REPORTS_DIR = ''

def query_to_df(query, db_dict, verbose=False):
    cnx = mysql.connector.connect(user=db_dict['USER'], password=db_dict['PASSWORD'],
                          host=db_dict['HOST'],
                          database=db_dict['DATABASE_NAME'])
    if verbose == True:
        print("connected to server")
    cursor = cnx.cursor()
    if verbose == True:
        print("cursor has been defined, now executing query: ", query)
    cursor.execute(query)
    if verbose == True:
        print("query has been executed, cursor is fetching all.")
    rows = cursor.fetchall()
    if verbose == True:
        print("rows have been fetched")
    df= pd.DataFrame(data = rows, columns = [i[0] for i in cursor.description]) 
    df=data_format(df)
    cnx.close()
    try:
        if df['page'][0][0:4]=='http':
            df['page'] = df['page'].apply(lambda x : x[28:])
    except:
        pass
    return df

def data_format(df):
    for col in df.columns:
    #this is because I don't have a function to convert the mysql data type to a pandas one!
        try:
            if col == 'clicks':
                df[col] = df[col].apply(lambda x : int(x))
            else:
                df[col] = df[col].apply(lambda x : float(x))
        except:
            pass
    return df

def recent_low_performers(table_name, start_date):

    print('pulling records from {} onwards'.format(start_date))
    
    recent_low_performers = r'''SELECT * FROM (SELECT page, sum(impressions) as impressions, sum(clicks) AS clicks, 
        AVG(ctr) AS avg_ctr, avg(position) as avg_position
    FROM {0} WHERE date > "{1}"  GROUP BY page) as s
    WHERE s.clicks < 100 and s.avg_position > 10 and s.impressions >100
    GROUP BY s.page
    ORDER BY s.impressions DESC
    LIMIT 250;'''.format(table_name, start_date)
    return query_to_df(recent_low_performers, db_dict)

def low_performers_report(db_dict, tables = ['acc_web','acc_image','acc_video'], reports_dir = 
                        REPORTS_DIR, get_queries=True, days_period=30):
    df_list=[]
    
    end_date=datetime.today()-timedelta(3)
    start_date=end_date- timedelta(days_period)
    end_date=datetime.strftime(end_date,'%Y-%m-%d')
    start_date=datetime.strftime(start_date,'%Y-%m-%d')
    print('start date is ', start_date)
    print('end date is ', end_date)
    
    for table in tables:
        df=recent_low_performers(table, start_date)
        df=data_format(df)
        df_list.append(df)
        
    report_name= reports_dir+'low performers report {}.xlsx'.format(end_date)
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
    
    for i in range(len(tables)):
        if tables[i][:4]=='acc_':
            tables[i]=tables[i][4:]
            
    if get_queries == True:

        #having removed 'acc_' from table names, the resulting table name refers to a table with query data
        for i in range(len(df_list)):
            qdf=query_report(df_list[i],db_dict, table=tables[i], start_date=start_date, end_date = end_date)
            print('appending qdf ', tables[i], ' to df_list')
            df_list.append(qdf)
            tables.append(tables[i] + "_queries")


    for i in range(len(tables)):
        # Convert the dataframe to an XlsxWriter Excel object.
        df_list[i].to_excel(writer, sheet_name = tables[i], index=False)
        
        #workbook  = writer.book
        
        worksheet = writer.sheets[tables[i]]
        for j in range(len(df_list[i].columns)):
            worksheet.set_column(j, j, len(df_list[i].columns[j]))
        
    writer.save()
    writer.close() #if you don't close it, it stays open via ipynb until you restart kernel.
    return
    
def query_report(df,db_dict, table='web',impressions_col='impressions', start_date='', end_date = '',
                 page_col='page', date_col = 'date', site='https://whoareyoumadeof.com/'):
    #Returns a df listing all queries for which the pages with over 1k impressions rank, grouped by page, query.
    #You can restrict the dates, start_date inclusive, end_date exclusive.
    #start_date, end_date, page_col, impressions_col are strings. eg: '2020-01-01', 'page', and 'sum_impressions'
    page_string = ''
    
    for page in df[page_col]:
        page_string += '''"{}",'''.format(site+page)
    page_string = page_string[:-1]
    
    if any([start_date, end_date]):
        date_restriction = "AND " 
    if all([start_date, end_date]):
        date_restriction +=  " date >= '" + start_date + "' and " + "date < '"+ end_date + "'"
    elif any([start_date, end_date]):
        date_restriction += " date >= {}".format(start_date) + "date < "+ end_date
    else:
        date_restriction = ""

    df = df[df[impressions_col]>=1000]
    #for each page receiving more than 1k impressions, include it in a list
    query_report='''SELECT page, query, sum(impressions) as impressions, sum(clicks) as clicks, avg(position) as avg_position 
        FROM {0}
        WHERE page in ({1}) {2}
        GROUP BY page, query ORDER BY page, impressions DESC;'''.format(table, page_string, date_restriction)
    
    query_df=query_to_df(query_report, db_dict)
    return query_df[query_df['impressions']>10]
    